# 연습문제: 합성곱 신경망 - MNIST

## 문제 1

MNIST 데이터셋을 이용하여 최고의 성능을 발휘하는 합성곱신경망 모델(CNN)을 구현하라.

In [ ]:
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np
import keras
import sys

### **convnet 모델 구성**

   1. **conv2d (Conv2D)** : 32개의 필터 커널 Convolutional Layer 2개
   2. **max_pooling2d (MaxPooling2D)** : `pool_size = (2,2)`, `stride = (1,1)`
   3. **conv2d (Conv2D)** : 64개의 필터 커널 Convolutional Layer 2개
   4. **max_pooling2d (MaxPooling2D)** : `pool_size = (2,2)`, `stride = (1,1)`
   5. **dropout (Dropout)** : 첫 번째 `Dropout` 층 0.25 설정
   6. **dense (Dense)** : `ReLU` 층 1024 설정, MNIST 데이터 평가를 위해 데이터 형식 변경
   7. **dropout (Dropout)** : 두 번째 `Dropout` 층 0.5 설정
   8. **dense (Dense)** : `Softmax` 층 10 설정, MNIST 데이터 평가를 위해 다중 클래스 분류 함수 `Softmax` 함수 설정

**모델 구성 요약**

In [ ]:
model_cnn = Sequential()
model_cnn.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), padding='same',
                 activation='relu',
                 input_shape=(28, 28, 1)))
model_cnn.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), padding='same',
                 activation='relu',
                 input_shape=(28, 28, 1)))
model_cnn.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1)))
model_cnn.add(Conv2D(64, (2, 2), activation='relu', padding='same'))
model_cnn.add(Conv2D(64, (2, 2), activation='relu', padding='same'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(Dropout(0.25))
model_cnn.add(Flatten())
model_cnn.add(Dense(1024, activation='relu'))
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(10, activation='softmax'))
model_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        832       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        25632     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 27, 27, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 27, 64)        8256      
                                                                 
 conv2d_3 (Conv2D)           (None, 27, 27, 64)        16448     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 64)       0         
 2D)                                                    

**MNIST 이미지 분류 훈련**

설정한 모델로 이미지 분류 훈련 진행

   - **데이터 세트 :** 검증:훈련:테스트 = 10000:50000:10000
   - **배치 사이즈 :** 64
   - **훈련 횟수 :** 10 회
   - **optimizer :** `rmsprop`에서 `adam`으로 변경

In [ ]:
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

11501568/11490434 [==============================] - 0s 0us/step


In [ ]:
print('train_images shape:', train_images.shape)
print('test_images shape:', test_images.shape)
print('val_images shape:', test_images.shape)

train_images shape: (50000, 28, 28, 1)
test_images shape: (10000, 28, 28, 1)
val_images shape: (10000, 28, 28, 1)


In [ ]:
model_cnn.compile(optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

model_cnn.fit(train_images, train_labels,
                 batch_size=64,
                 epochs=10,
                 verbose=1, 
                 validation_data=(val_images, val_labels))

Epoch 1/10
782/782 [==============================] - 52s 27ms/step - loss: 0.1362 - accuracy: 0.9581 - val_loss: 0.0528 - val_accuracy: 0.9843
Epoch 2/10
782/782 [==============================] - 21s 27ms/step - loss: 0.0512 - accuracy: 0.9844 - val_loss: 0.0446 - val_accuracy: 0.9861
Epoch 3/10
782/782 [==============================] - 21s 27ms/step - loss: 0.0390 - accuracy: 0.9878 - val_loss: 0.0447 - val_accuracy: 0.9862
Epoch 4/10
782/782 [==============================] - 21s 27ms/step - loss: 0.0326 - accuracy: 0.9898 - val_loss: 0.0344 - val_accuracy: 0.9896
Epoch 5/10
782/782 [==============================] - 21s 26ms/step - loss: 0.0254 - accuracy: 0.9920 - val_loss: 0.0401 - val_accuracy: 0.9898
Epoch 6/10
782/782 [==============================] - 21s 26ms/step - loss: 0.0239 - accuracy: 0.9925 - val_loss: 0.0346 - val_accuracy: 0.9910
Epoch 7/10
782/782 [==============================] - 21s 26ms/step - loss: 0.0190 - accuracy: 0.9939 - val_loss: 0.0368 - val_accuracy:

**훈련된 convnet 평가**

테스트셋에 대한 성능이 7 - 8장에서 보여준 모델보다 좋은 평가를 보여준다.

약 20번 정도 훈련을 초기화하면서 시도해봤는데, 테스트 셋에 대해 `accuracy` = **99.10 ~ 99.45** 정도의 정확도를 보여준다.

In [ ]:
test_metrics = model_cnn.evaluate(test_images, test_labels)
predictions = model_cnn.predict(test_images)

313/313 [==============================] - 2s 6ms/step - loss: 0.0214 - accuracy: 0.9941


In [ ]:
test_loss, test_acc = model_cnn.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc:.3f}")

313/313 [==============================] - 2s 6ms/step - loss: 0.0214 - accuracy: 0.9941
Test accuracy: 0.994
